# Etapa 1 - Ver tablas 
**Inspeccion rapida del .sqlite**

In [ ]:
import os 
import sqlite3

# Ruta de la base de datos SQLite
BASE_DIR = os.path.dirname(os.getcwd())  # Sube desde /notebooks a la raíz
db_path = os.path.join(BASE_DIR, "data", "raw", "2019-12.sqlite")

# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Litar las tablas existentes 
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tablas en la base de datos:")
for t in tables:
    print("-", t[0])

# Cerrar la conexión
conn.close()


# Etapa 2 - Ver columnas y tipos
**Revisamos la estructura de la tabla**


In [ ]:
import os 
import sqlite3

# Ruta de la base de datos SQLite
BASE_DIR = os.path.dirname(os.getcwd())  # Sube desde /notebooks a la raíz
db_path = os.path.join(BASE_DIR, "data", "raw", "2019-12.sqlite")
# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Ver columnas de la tabla tripdata
cursor.execute("PRAGMA table_info(tripdata);")
cols = cursor.fetchall()

print("Columnas en tripdata:")
for col in cols:
    # col = (cid, name, type, notnull, dflt_value, pk)
    print(f"- {col[1]} ({col[2]})")

conn.close()

# Etapa 3 - Revisar Zonas 
**Para saber como estan codificadas las zonas**


In [ ]:
import os 
import pandas as pd

# Ruta del archivo de zonas 
BASE_DIR = os.path.dirname(os.getcwd())  
zones_path = os.path.join(BASE_DIR, "data", "taxi_zone_lookup.csv")

# Leer el archivo CSV
zones = pd.read_csv(zones_path)

# 2️⃣ Revisar estructura
print("\n📌 Primeras filas de taxi_zone_lookup.csv:")
print(zones.head())

print("\n📋 Columnas disponibles:")
print(zones.columns.tolist())

# 3️⃣ Revisar tipos de datos
print("\n🧾 Tipos de datos:")
print(zones.dtypes)

# 4️⃣ Comprobar si hay nulos o duplicados
print("\n❓ Nulos por columna:")
print(zones.isna().sum())

print("\n🔁 Duplicados por LocationID:")
print(zones['LocationID'].duplicated().sum())


In [ ]:
import os 
# ✅ Etapa 3: Preparar tabla de zonas (lookup)
BASE_DIR = os.path.dirname(os.getcwd())  
zones_path = os.path.join(BASE_DIR, "data", "taxi_zone_lookup.csv")
zones = pd.read_csv(zones_path)

# Nos quedamos con las columnas necesarias
zones = zones[['LocationID', 'Borough', 'Zone']]

# Verificamos el resultado final
print("\n✅ Tabla de zonas (lookup) lista:")
print(zones.head(10).to_string(index=False))

# Etapa 4 - Exploracion basica y conteos
**Top zonas pickup/dropoff con nombres**


In [ ]:
import sqlite3
import pandas as pd

# Ruta de la base de datos SQLite
BASE_DIR = os.path.dirname(os.getcwd())  # Sube desde /notebooks a la raíz
db_path = os.path.join(BASE_DIR, "data", "raw", "2019-12.sqlite")

# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)

# Leemos una muestra de datos de la tabla tripdata
query = "SELECT * FROM tripdata LIMIT 100000;"
df = pd.read_sql_query(query, conn)

# Cerramos la conexión
conn.close()

# Verificamos la estructura del DataFrame
print("\n📌 Columnas disponibles en tripdata:")
print(df.columns.tolist())

print("\n Primeras filas de tripdata:")
print(df.head())


In [ ]:
# 🧼 Convertimos pulocationid y dolocationid a enteros
df['pulocationid'] = df['pulocationid'].astype(int)
df['dolocationid'] = df['dolocationid'].astype(int)

# 👀 Verificamos rápidamente
print(df[['pulocationid', 'dolocationid']].head())
print(df[['pulocationid', 'dolocationid']].dtypes)

In [ ]:
# Conteos Top de zonas de pickup y dropoff

# Top zonas de recogida (pickup)
pickup_counts = (
    df['pulocationid']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'LocationID', 'pulocationid': 'LocationID'})
)

# Top zonas de entrega (dropoff)
dropoff_counts = (
    df['dolocationid']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'LocationID', 'dolocationid': 'LocationID'})
)

print("🚖 Top 15 zonas de recogida (pickup) [IDs]:")
print(pickup_counts.head(15).to_string(index=False))

print("\n📍 Top 15 zonas de entrega (dropoff) [IDs]:")
print(dropoff_counts.head(15).to_string(index=False))